# School District Analysis

![Books Image](images/education.jpg)

In this demonstration, a dataset of high school students' math and reading scores is analyzed to help make budget decisions for a school district.
## Goals:

#### Will create a high level snapshot (in table form) of the district's key metrics, including:


- Total Schools
- Total Students
- Total Budget
- Average Math Score
- Average Reading Score
- % Passing Math
- % Passing Reading
- Overall Passing Rate (Average of the above two)




#### School Summary


Will create an overview table that summarizes key metrics about each school, including:


- School Name
- School Type
- Total Students
- Total School Budget
- Per School Budget
- Average Math Score
- Average Reading Score
- % Passing Math
- % Passing Reading
- Overall Passing Rate (Average of the above two)




#### Top Performing Schools (By Passing Rate)


Will create a table that highlights the top 5 performing schools based on Overall Passing Rate. Will include:


- School Name
- School Type
- Total Students
- Total School Budget
- Per School Budget
- Average Math Score
- Average Reading Score
- % Passing Math
- % Passing Reading
- Overall Passing Rate (Average of the above two)




#### Top Performing Schools (By Passing Rate)


Will create a table that highlights the bottom 5 performing schools based on Overall Passing Rate. Will include all of the same metrics as above.


#### Math Scores by Grade


Will create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.


#### Reading Scores by Grade


Will create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.


#### Scores by School Spending


Will create a table that breaks down school performances based on average Spending Ranges (Per Student). Will use 4 reasonable bins to group school spending. Will include in the table each of the following:


- Average Math Score
- Average Reading Score
- % Passing Math
- % Passing Reading
- Overall Passing Rate (Average of the above two)




#### Scores by School Size


- Will repeat the above breakdown, but this time will group schools based on a reasonable approximation of school size (Small, Medium, Large).


#### Scores by School Type


- Will repeat the above breakdown, but this time group schools based on school type (Charter vs. District).


#### As final considerations:


- The script will work for both datasets.
- Will comment on observable trends in the data.

#### Import dependencies

In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Read in CSVs

In [106]:
schools_csv = pd.read_csv('../pandas_homework/inputfiles/schools_complete.csv')
students_csv = pd.read_csv('../pandas_homework/inputfiles/students_complete.csv')
schools = pd.DataFrame(schools_csv)
students = pd.DataFrame(students_csv)

#### Explore data

In [107]:
schools.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [108]:
students.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


### High Level Snapshot

#### Perform summary calculations and store

In [109]:
schoolnum = len(schools['School ID'].unique())

In [110]:
studentnum = len(students['Student ID'].unique())

In [111]:
totbudget = schools.budget.sum()

In [112]:
avmath = students.math_score.mean()

In [113]:
avreading = students.reading_score.mean()

In [114]:
passing_reading = students[students['reading_score'] >= 70]
passr = passing_reading['Student ID'].count()

In [115]:
passing_math = students[students['math_score'] >= 70]
passm = passing_math['Student ID'].count()

In [116]:
readingpassrate = (passr / studentnum) * 100

In [117]:
mathpassrate = (passm / studentnum) * 100

In [118]:
totpassrate = (readingpassrate + mathpassrate) / 2

#### Create snapshot and format
Make sure to save unformatted snapshot as a variable in case we want to plot

In [119]:
snapshot = pd.DataFrame({'Total Schools': schoolnum,
                         'Total Students': studentnum,
                         'Total Budget': totbudget,
                         'Average Math Score': avmath,
                         'Average Reading Score': avreading,
                         '% Passing Math': mathpassrate,
                         '% Passing Reading': readingpassrate,
                         'Overall Passing Rate': totpassrate}, index=[0])

formatted_snapshot = snapshot.style.format({'Total Budget': '${:,.2f}',
                                            'Average Math Score': '{:.2f}%',
                                            'Average Reading Score': '{:.2f}%',
                                            '% Passing Math': '{:.2f}%',
                                            '% Passing Reading': '{:.2f}%',
                                            'Overall Passing Rate': '{:.2f}%'})
formatted_snapshot

,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Overall Passing Rate,Total Budget,Total Schools,Total Students
0,74.98%,85.81%,78.99%,81.88%,80.39%,"$24,649,428.00",15,39170


### School Summary

In [120]:
schools.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [121]:
students.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


Strategy:

1. First start with schools dataframe.
2. Then we need to merge different students groupby queries to schools, one at a time.

The first prepatory step will be to make sure that there is an identical column name between the two dataframes to serve as a key. This will allow us to avoid duplicate columns when merging.
- Lets rename the 'name' column in the schools dataframe to match the student's dataframe column name, 'schools'
- While we're at it, lets remove the superfluous 'School ID' column.

In [122]:
schools = schools.rename(columns={'name':'school'})

In [123]:
schools = schools.drop('School ID', axis=1)

Great, now we are ready to create a query from the groupby object and merge the result to the schools dataframe

First create a copy of schools:

In [124]:
schools_copy = schools

Then create the groupby object out of the students dataframe
- groupby school to get summaries for each school

In [125]:
schools_group = students.groupby('school')

Next, query the groupby object to return average reading score for each school
- This will create a series, with the schools as the index.
  - Therefore, because merge will not work with a series, we need to turn the result into a dataframe
    - We will also need to reset the index so we can call out the school column in the merge.

In [126]:
reading_query = pd.DataFrame(schools_group['reading_score'].mean()).reset_index()

Now we will merge on the common column:

In [127]:
step1 = schools_copy.merge(reading_query, on='school', how='inner')

And we will do this for math score:

In [128]:
math_query = pd.DataFrame(schools_group['math_score'].mean()).reset_index()
step2 = step1.merge(math_query, on='school', how='inner')

Now we will do some calculated queries:
- the % students that passed math
- the % students that passed reading

#### Enter ```.apply``` and the ```lambda``` function:

```python
schools_group.apply(lambda x:np.mean(x['math_score']))
```

is the same as

```python
schools_group['math_score'].mean()
```

the benefit here is that you can write any function now and return the result inside a groupby.
- just replace ```np.mean()``` with a defined function

In [129]:
def countpass(series):
    counter = 0
    for x in series:
        if x >= 70:
            counter = counter + 1
    return counter
        

In [130]:
mathpass_query = pd.DataFrame(schools_group.apply(lambda x:countpass(x['math_score']))).reset_index()
readingpass_query = pd.DataFrame(schools_group.apply(lambda x:countpass(x['reading_score']))).reset_index()

Now we will merge these queries with the dataframe we are building on:

In [131]:
step3 = step2.merge(mathpass_query, on='school', how='inner')
step4 = step3.merge(readingpass_query, on='school', how='inner')
step4

,school,type,size,budget,reading_score,math_score,0_x,0_y
0,Huang High School,District,2917,1910635,81.182722,76.629414,1916,2372
1,Figueroa High School,District,2949,1884411,81.158020,76.711767,1946,2381
2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,1653,1688
3,Hernandez High School,District,4635,3022020,80.934412,77.289752,3094,3748
4,Griffin High School,Charter,1468,917500,83.816757,83.351499,1371,1426
5,Wilson High School,Charter,2283,1319574,83.989488,83.274201,2143,2204
6,Cabrera High School,Charter,1858,1081356,83.975780,83.061895,1749,1803
7,Bailey High School,District,4976,3124928,81.033963,77.048432,3318,4077
8,Holden High School,Charter,427,248087,83.814988,83.803279,395,411
9,Pena High School,Charter,962,585858,84.044699,83.839917,910,923


Checking math...:

In [132]:
students[(students['school'] == 'Bailey High School') & (students['math_score'] >= 70)]['Student ID'].count()

3318

Looks good, now that all of the groupby queries are done, we need to add some calculated columns to the dataframe.

Right now we have the number of students that passed math and reading respectively.
- We need to calculate percentages of students that passed for math and reading respectively
- we then need to average out those two numbers to give an overall pass rate for each school.

In [133]:
step4['% Passing Math'] = step4['0_x'] / step4['size'] * 100
step4['% Passing Reading'] = step4['0_y'] / step4['size'] * 100
step4['% Passing Overall'] = (step4['% Passing Math'] + step4['% Passing Reading']) / 2
step4

,school,type,size,budget,reading_score,math_score,0_x,0_y,% Passing Math,% Passing Reading,% Passing Overall
0,Huang High School,District,2917,1910635,81.182722,76.629414,1916,2372,65.683922,81.316421,73.500171
1,Figueroa High School,District,2949,1884411,81.158020,76.711767,1946,2381,65.988471,80.739234,73.363852
2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,1653,1688,93.867121,95.854628,94.860875
3,Hernandez High School,District,4635,3022020,80.934412,77.289752,3094,3748,66.752967,80.862999,73.807983
4,Griffin High School,Charter,1468,917500,83.816757,83.351499,1371,1426,93.392371,97.138965,95.265668
5,Wilson High School,Charter,2283,1319574,83.989488,83.274201,2143,2204,93.867718,96.539641,95.203679
6,Cabrera High School,Charter,1858,1081356,83.975780,83.061895,1749,1803,94.133477,97.039828,95.586652
7,Bailey High School,District,4976,3124928,81.033963,77.048432,3318,4077,66.680064,81.933280,74.306672
8,Holden High School,Charter,427,248087,83.814988,83.803279,395,411,92.505855,96.252927,94.379391
9,Pena High School,Charter,962,585858,84.044699,83.839917,910,923,94.594595,95.945946,95.270270


We also need to add total budget for all schools:

In [134]:
step4['Total Budget'] = step4['budget'].sum()
step4

,school,type,size,budget,reading_score,math_score,0_x,0_y,% Passing Math,% Passing Reading,% Passing Overall,Total Budget
0,Huang High School,District,2917,1910635,81.182722,76.629414,1916,2372,65.683922,81.316421,73.500171,24649428
1,Figueroa High School,District,2949,1884411,81.158020,76.711767,1946,2381,65.988471,80.739234,73.363852,24649428
2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,1653,1688,93.867121,95.854628,94.860875,24649428
3,Hernandez High School,District,4635,3022020,80.934412,77.289752,3094,3748,66.752967,80.862999,73.807983,24649428
4,Griffin High School,Charter,1468,917500,83.816757,83.351499,1371,1426,93.392371,97.138965,95.265668,24649428
5,Wilson High School,Charter,2283,1319574,83.989488,83.274201,2143,2204,93.867718,96.539641,95.203679,24649428
6,Cabrera High School,Charter,1858,1081356,83.975780,83.061895,1749,1803,94.133477,97.039828,95.586652,24649428
7,Bailey High School,District,4976,3124928,81.033963,77.048432,3318,4077,66.680064,81.933280,74.306672,24649428
8,Holden High School,Charter,427,248087,83.814988,83.803279,395,411,92.505855,96.252927,94.379391,24649428
9,Pena High School,Charter,962,585858,84.044699,83.839917,910,923,94.594595,95.945946,95.270270,24649428


Now we need to:
- get rid of superfluous columns
- rename columns to be more intelligible
- Order the columns logically

In [135]:
reduced = step4.drop('0_x', axis=1)

In [136]:
reduced_more = reduced.drop('0_y', axis=1)

In [137]:
renamed = reduced_more.rename(columns={'school':'School',
                                       'type':'Type',
                                       'size':'Total Students',
                                       'budget':'School Budget',
                                       'reading_score':'Average Reading Score',
                                       'math_score':'Average Math Score'})

formatted = renamed.style.format({'School Budget':'${:,.0f}',
                                 'Total Budget':'${:,.0f}',
                                 'Average Reading Score':'{:.1f}%',
                                 'Average Math Score':'{:.1f}%',
                                 '% Passing Math':'{:.1f}%',
                                 '% Passing Reading':'{:.1f}%',
                                 '% Passing Overall':'{:.1f}%'})

In [138]:
formatted

,School,Type,Total Students,School Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Passing Overall,Total Budget
0,Huang High School,District,2917,"$1,910,635",81.2%,76.6%,65.7%,81.3%,73.5%,"$24,649,428"
1,Figueroa High School,District,2949,"$1,884,411",81.2%,76.7%,66.0%,80.7%,73.4%,"$24,649,428"
2,Shelton High School,Charter,1761,"$1,056,600",83.7%,83.4%,93.9%,95.9%,94.9%,"$24,649,428"
3,Hernandez High School,District,4635,"$3,022,020",80.9%,77.3%,66.8%,80.9%,73.8%,"$24,649,428"
4,Griffin High School,Charter,1468,"$917,500",83.8%,83.4%,93.4%,97.1%,95.3%,"$24,649,428"
5,Wilson High School,Charter,2283,"$1,319,574",84.0%,83.3%,93.9%,96.5%,95.2%,"$24,649,428"
6,Cabrera High School,Charter,1858,"$1,081,356",84.0%,83.1%,94.1%,97.0%,95.6%,"$24,649,428"
7,Bailey High School,District,4976,"$3,124,928",81.0%,77.0%,66.7%,81.9%,74.3%,"$24,649,428"
8,Holden High School,Charter,427,"$248,087",83.8%,83.8%,92.5%,96.3%,94.4%,"$24,649,428"
9,Pena High School,Charter,962,"$585,858",84.0%,83.8%,94.6%,95.9%,95.3%,"$24,649,428"


#### Top Performing Schools (By Passing Rate)

Create a table that highlights the top 5 performing schools based on Overall Passing Rate. Include:
- School Name
- School Type
- Total Students
- Total School Budget
- Per School Budget
- Average Math Score
- Average Reading Score
- % Passing Math
- % Passing Reading
- Overall Passing Rate (Average of the above two)

To accomplish all of this, all we need to do is sort the (unformatted) dataframe.

In [139]:
bypassrate = renamed.sort_values(by='% Passing Overall', ascending=False)

In [140]:
top_passing_schools = pd.DataFrame(bypassrate.head())

In [141]:
top_passing_schools

,School,Type,Total Students,School Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Passing Overall,Total Budget
6,Cabrera High School,Charter,1858,1081356,83.975780,83.061895,94.133477,97.039828,95.586652,24649428
14,Thomas High School,Charter,1635,1043130,83.848930,83.418349,93.272171,97.308869,95.290520,24649428
9,Pena High School,Charter,962,585858,84.044699,83.839917,94.594595,95.945946,95.270270,24649428
4,Griffin High School,Charter,1468,917500,83.816757,83.351499,93.392371,97.138965,95.265668,24649428
5,Wilson High School,Charter,2283,1319574,83.989488,83.274201,93.867718,96.539641,95.203679,24649428


We can format this using the above code:

In [142]:
formatted_2 = top_passing_schools.style.format({'School Budget':'${:,.0f}',
                                                'Total Budget':'${:,.0f}',
                                                'Average Reading Score':'{:.1f}%',
                                                'Average Math Score':'{:.1f}%',
                                                '% Passing Math':'{:.1f}%',
                                                '% Passing Reading':'{:.1f}%',
                                                '% Passing Overall':'{:.1f}%'})

formatted_2

,School,Type,Total Students,School Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Passing Overall,Total Budget
6,Cabrera High School,Charter,1858,"$1,081,356",84.0%,83.1%,94.1%,97.0%,95.6%,"$24,649,428"
14,Thomas High School,Charter,1635,"$1,043,130",83.8%,83.4%,93.3%,97.3%,95.3%,"$24,649,428"
9,Pena High School,Charter,962,"$585,858",84.0%,83.8%,94.6%,95.9%,95.3%,"$24,649,428"
4,Griffin High School,Charter,1468,"$917,500",83.8%,83.4%,93.4%,97.1%,95.3%,"$24,649,428"
5,Wilson High School,Charter,2283,"$1,319,574",84.0%,83.3%,93.9%,96.5%,95.2%,"$24,649,428"


#### Bottom performing schools:

Will do the same for bottom five schools based on % passing overall

In [143]:
bypassrate_asc = renamed.sort_values(by='% Passing Overall')
bottom_passing_schools = pd.DataFrame(bypassrate_asc.head())


formatted_3 = bottom_passing_schools.style.format({'School Budget':'${:,.0f}',
                                                   'Total Budget':'${:,.0f}',
                                                   'Average Reading Score':'{:.1f}%',
                                                   'Average Math Score':'{:.1f}%',
                                                   '% Passing Math':'{:.1f}%',
                                                   '% Passing Reading':'{:.1f}%',
                                                   '% Passing Overall':'{:.1f}%'})
formatted_3

,School,Type,Total Students,School Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Passing Overall,Total Budget
11,Rodriguez High School,District,3999,"$2,547,363",80.7%,76.8%,66.4%,80.2%,73.3%,"$24,649,428"
1,Figueroa High School,District,2949,"$1,884,411",81.2%,76.7%,66.0%,80.7%,73.4%,"$24,649,428"
0,Huang High School,District,2917,"$1,910,635",81.2%,76.6%,65.7%,81.3%,73.5%,"$24,649,428"
12,Johnson High School,District,4761,"$3,094,650",81.0%,77.1%,66.1%,81.2%,73.6%,"$24,649,428"
13,Ford High School,District,2739,"$1,763,916",80.7%,77.1%,68.3%,79.3%,73.8%,"$24,649,428"


#### Math Scores by Grade
 - Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

This will be a bit easier because it is asking for only a scalar summary for 2 nested groups.

In [144]:
answer1 = pd.DataFrame(students.groupby(['school', 'grade'])['math_score'].mean())

This is the same as:

```python
group = students.groupby(['school', 'grade'])
returned = group['math_score'].mean()
answer1 = pd.DataFrame(returned)
```

In [145]:
answer1

math_score
school                grade            
Bailey High School    10th    76.996772
                      11th    77.515588
                      12th    76.492218
                      9th     77.083676
Cabrera High School   10th    83.154506
                      11th    82.765560
                      12th    83.277487
                      9th     83.094697
Figueroa High School  10th    76.539974
                      11th    76.884344
                      12th    77.151369
                      9th     76.403037
Ford High School      10th    77.672316
                      11th    76.918058
                      12th    76.179963
                      9th     77.361345
Griffin High School   10th    84.229064
                      11th    83.842105
                      12th    83.356164
                      9th     82.044010
Hernandez High School 10th    77.337408
                      11th    77.136029
                      12th    77.186567
                      9th     77.438495
Holden High School    10th    83.429825
                      11th    85.000000
                      12th    82.855422
                      9th     83.787402
Huang High School     10th    75.908735
                      11th    76.446602
                      12th    77.225641
                      9th     77.027251
Johnson High School   10th    76.691117
                      11th    77.491653
                      12th    76.863248
                      9th     77.187857
Pena High School      10th    83.372000
                      11th    84.328125
                      12th    84.121547
                      9th     83.625455
Rodriguez High School 10th    76.612500
                      11th    76.395626
                      12th    77.690748
                      9th     76.859966
Shelton High School   10th    82.917411
                      11th    83.383495
                      12th    83.778976
                      9th     83.420755
Thomas High School    10th    83.087886
                      11th    83.498795
                      12th    83.497041
                      9th     83.590022
Wilson High School    10th    83.724422
                      11th    83.195326
                      12th    83.035794
                      9th     83.085578
Wright High School    10th    84.010288
                      11th    83.836782
                      12th    83.644986
                      9th     83.264706

We can change the order of displayed grades by using the ```.reindex()``` function on the groupby result:

In [146]:
answer1 = answer1.reindex(['9th', '10th', '11th', '12th'], level='grade')
answer1

math_score
school                grade            
Bailey High School    9th     77.083676
                      10th    76.996772
                      11th    77.515588
                      12th    76.492218
Cabrera High School   9th     83.094697
                      10th    83.154506
                      11th    82.765560
                      12th    83.277487
Figueroa High School  9th     76.403037
                      10th    76.539974
                      11th    76.884344
                      12th    77.151369
Ford High School      9th     77.361345
                      10th    77.672316
                      11th    76.918058
                      12th    76.179963
Griffin High School   9th     82.044010
                      10th    84.229064
                      11th    83.842105
                      12th    83.356164
Hernandez High School 9th     77.438495
                      10th    77.337408
                      11th    77.136029
                      12th    77.186567
Holden High School    9th     83.787402
                      10th    83.429825
                      11th    85.000000
                      12th    82.855422
Huang High School     9th     77.027251
                      10th    75.908735
                      11th    76.446602
                      12th    77.225641
Johnson High School   9th     77.187857
                      10th    76.691117
                      11th    77.491653
                      12th    76.863248
Pena High School      9th     83.625455
                      10th    83.372000
                      11th    84.328125
                      12th    84.121547
Rodriguez High School 9th     76.859966
                      10th    76.612500
                      11th    76.395626
                      12th    77.690748
Shelton High School   9th     83.420755
                      10th    82.917411
                      11th    83.383495
                      12th    83.778976
Thomas High School    9th     83.590022
                      10th    83.087886
                      11th    83.498795
                      12th    83.497041
Wilson High School    9th     83.085578
                      10th    83.724422
                      11th    83.195326
                      12th    83.035794
Wright High School    9th     83.264706
                      10th    84.010288
                      11th    83.836782
                      12th    83.644986

Now we need to:
- rename columns
- format math_score

In [147]:
answer2 = answer1.reset_index()
answer2

,school,grade,math_score
0,Bailey High School,9th,77.083676
1,Bailey High School,10th,76.996772
2,Bailey High School,11th,77.515588
3,Bailey High School,12th,76.492218
4,Cabrera High School,9th,83.094697
5,Cabrera High School,10th,83.154506
6,Cabrera High School,11th,82.765560
7,Cabrera High School,12th,83.277487
8,Figueroa High School,9th,76.403037
9,Figueroa High School,10th,76.539974


In [148]:
answer3 = answer2.rename(columns={'school':'School',
                                  'grade':'Grade',
                                  'math_score':'Average Math Score'})
answer3

,School,Grade,Average Math Score
0,Bailey High School,9th,77.083676
1,Bailey High School,10th,76.996772
2,Bailey High School,11th,77.515588
3,Bailey High School,12th,76.492218
4,Cabrera High School,9th,83.094697
5,Cabrera High School,10th,83.154506
6,Cabrera High School,11th,82.765560
7,Cabrera High School,12th,83.277487
8,Figueroa High School,9th,76.403037
9,Figueroa High School,10th,76.539974


In [149]:
answer4 = answer3.style.format({'Average Math Score':'{:.1f}%'})
answer4

,School,Grade,Average Math Score
0,Bailey High School,9th,77.1%
1,Bailey High School,10th,77.0%
2,Bailey High School,11th,77.5%
3,Bailey High School,12th,76.5%
4,Cabrera High School,9th,83.1%
5,Cabrera High School,10th,83.2%
6,Cabrera High School,11th,82.8%
7,Cabrera High School,12th,83.3%
8,Figueroa High School,9th,76.4%
9,Figueroa High School,10th,76.5%


#### Reading scores by grade:
We will do the same for reading scores as above:

In [150]:
answer5 = pd.DataFrame(students.groupby(['school', 'grade'])['reading_score'].mean())
answer5 = answer5.reindex(['9th', '10th', '11th', '12th'], level='grade')
answer6 = answer5.reset_index()
answer7 = answer6.rename(columns={'school':'School',
                                  'grade':'Grade',
                                  'reading_score':'Average Reading Score'})
answer8 = answer7.style.format({'Average Reading Score':'{:.1f}%'})
answer8

,School,Grade,Average Reading Score
0,Bailey High School,9th,81.3%
1,Bailey High School,10th,80.9%
2,Bailey High School,11th,80.9%
3,Bailey High School,12th,80.9%
4,Cabrera High School,9th,83.7%
5,Cabrera High School,10th,84.3%
6,Cabrera High School,11th,83.8%
7,Cabrera High School,12th,84.3%
8,Figueroa High School,9th,81.2%
9,Figueroa High School,10th,81.4%


#### Scores by School Spending

Will create a table that breaks down school performances based on average Spending Ranges (Per Student). Will use 4 reasonable bins to group school spending. Will include in the table each of the following:
- Average Math Score
- Average Reading Score
- % Passing Math
- % Passing Reading
- Overall Passing Rate (Average of the above two)

In [151]:
per_student = pd.DataFrame({'School': schools.school,
                            'Per Student Spending': schools.budget / schools.size})

In [152]:
per_student

,Per Student Spending,School
0,31843.916667,Huang High School
1,31406.850000,Figueroa High School
2,17610.000000,Shelton High School
3,50367.000000,Hernandez High School
4,15291.666667,Griffin High School
5,21992.900000,Wilson High School
6,18022.600000,Cabrera High School
7,52082.133333,Bailey High School
8,4134.783333,Holden High School
9,9764.300000,Pena High School


In [153]:
per_student_sorted = per_student.sort_values(by='Per Student Spending', ascending=False)

In [154]:
pssmax = per_student_sorted['Per Student Spending'].max()

In [155]:
pssmin = per_student_sorted['Per Student Spending'].min()

In [156]:
pssrng = pssmax - pssmin

In [157]:
# calculate bin width by dividing range into number of desired bins
binsiz = pssrng / 4

In [158]:
binsiz

11986.8375

In [159]:
# create array containing bin borders
bins = np.arange(pssmin, pssmax + binsiz, binsiz)

In [160]:
bins

array([  4134.78333333,  16121.62083333,  28108.45833333,  40095.29583333,
        52082.13333333])

In [161]:
labels = ['Low', 'Below Average', 'Above Average', 'High']

In [162]:
# label spending array with bin array
binned = pd.cut(per_student_sorted['Per Student Spending'], bins, labels=labels, include_lowest=True)

In [164]:
binned = pd.DataFrame(binned)


In [165]:
# merge bin labels with dataframe
withbins = per_student_sorted.merge(binned, how='inner', left_index=True, right_index=True)

In [166]:
withbins

,Per Student Spending_x,School,Per Student Spending_y
7,52082.133333,Bailey High School,High
12,51577.500000,Johnson High School,High
3,50367.000000,Hernandez High School,High
11,42456.050000,Rodriguez High School,High
0,31843.916667,Huang High School,Above Average
1,31406.850000,Figueroa High School,Above Average
13,29398.600000,Ford High School,Above Average
5,21992.900000,Wilson High School,Below Average
6,18022.600000,Cabrera High School,Below Average
2,17610.000000,Shelton High School,Below Average


Ok, now that we have the spending bin for each school, we should:
- remove the per student spending x column
- join this on 'school' to the students dataset
- groupby the per_student_spending bin


In [167]:
withbins = withbins.drop('Per Student Spending_x', axis=1)

In [168]:
withbins

,School,Per Student Spending_y
7,Bailey High School,High
12,Johnson High School,High
3,Hernandez High School,High
11,Rodriguez High School,High
0,Huang High School,Above Average
1,Figueroa High School,Above Average
13,Ford High School,Above Average
5,Wilson High School,Below Average
6,Cabrera High School,Below Average
2,Shelton High School,Below Average


In [169]:
renamed.head()

,School,Type,Total Students,School Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Passing Overall,Total Budget
0,Huang High School,District,2917,1910635,81.182722,76.629414,65.683922,81.316421,73.500171,24649428
1,Figueroa High School,District,2949,1884411,81.158020,76.711767,65.988471,80.739234,73.363852,24649428
2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,93.867121,95.854628,94.860875,24649428
3,Hernandez High School,District,4635,3022020,80.934412,77.289752,66.752967,80.862999,73.807983,24649428
4,Griffin High School,Charter,1468,917500,83.816757,83.351499,93.392371,97.138965,95.265668,24649428


In [170]:
# join bin labels to large dataframe
joined = renamed.merge(withbins, on='School', how='inner')

In [171]:
joined.head(20)

,School,Type,Total Students,School Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Passing Overall,Total Budget,Per Student Spending_y
0,Huang High School,District,2917,1910635,81.182722,76.629414,65.683922,81.316421,73.500171,24649428,Above Average
1,Figueroa High School,District,2949,1884411,81.158020,76.711767,65.988471,80.739234,73.363852,24649428,Above Average
2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,93.867121,95.854628,94.860875,24649428,Below Average
3,Hernandez High School,District,4635,3022020,80.934412,77.289752,66.752967,80.862999,73.807983,24649428,High
4,Griffin High School,Charter,1468,917500,83.816757,83.351499,93.392371,97.138965,95.265668,24649428,Low
5,Wilson High School,Charter,2283,1319574,83.989488,83.274201,93.867718,96.539641,95.203679,24649428,Below Average
6,Cabrera High School,Charter,1858,1081356,83.975780,83.061895,94.133477,97.039828,95.586652,24649428,Below Average
7,Bailey High School,District,4976,3124928,81.033963,77.048432,66.680064,81.933280,74.306672,24649428,High
8,Holden High School,Charter,427,248087,83.814988,83.803279,92.505855,96.252927,94.379391,24649428,Low
9,Pena High School,Charter,962,585858,84.044699,83.839917,94.594595,95.945946,95.270270,24649428,Low


In [172]:
# isolate pertinent columns
subset = joined[['Per Student Spending_y',
                 'Average Math Score', 
                 'Average Reading Score', 
                 '% Passing Math', 
                 '% Passing Reading', 
                 '% Passing Overall']]

In [173]:
# groupby bins for summary
spendgrp = subset.groupby('Per Student Spending_y')

In [174]:
# calculate averages and display
output1 = spendgrp.mean()
output1

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Per Student Spending_y,,,,,
Low,83.664898,83.892148,93.497607,96.445946,94.971776
Below Average,83.359224,83.898984,93.694764,96.670815,95.182790
Above Average,76.814591,81.029000,66.660665,80.451556,73.556111
High,77.063340,80.919864,66.464293,81.059691,73.761992


In [175]:
# format for readability
outputformatted = output1.style.format({'Average Math Score': '{:.1f}',
                                        'Average Reading Score': '{:.1f}',
                                        '% Passing Math': '{:.1f}%',
                                        '% Passing Reading': '{:.1f}%',
                                        '% Passing Overall': '{:.1f}%'})

outputformatted

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Per Student Spending_y,,,,,
Low,83.7,83.9,93.5%,96.4%,95.0%
Below Average,83.4,83.9,93.7%,96.7%,95.2%
Above Average,76.8,81.0,66.7%,80.5%,73.6%
High,77.1,80.9,66.5%,81.1%,73.8%


#### Performance by school size

In [176]:
sizemax = schools['size'].max()
sizemax

4976

In [177]:
sizemin = schools['size'].min()
sizemin

427

In [178]:
sizerang = sizemax - sizemin
sizerang

4549

In [179]:
# calculate bin width as range divided by number of desired bins
binsize = sizerang / 3
binsize

1516.3333333333333

In [180]:
# create bin border array
bins = np.arange(sizemin, sizemax + binsize, binsize)

In [181]:
bins

array([  427.        ,  1943.33333333,  3459.66666667,  4976.        ])

In [182]:
labels = ['Small', 'Medium', 'Large']

In [183]:
# label school size with bins
bysize = pd.cut(schools['size'], bins, labels=labels, include_lowest=True)
bysize = pd.DataFrame(bysize)
bysize

,size
0,Medium
1,Medium
2,Small
3,Large
4,Small
5,Medium
6,Small
7,Large
8,Small
9,Small


In [184]:
# join bin labels back to large dataframe
withsize = renamed.merge(bysize, left_index=True, right_index=True)

In [186]:
withsize

,School,Type,Total Students,School Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Passing Overall,Total Budget,size
0,Huang High School,District,2917,1910635,81.182722,76.629414,65.683922,81.316421,73.500171,24649428,Medium
1,Figueroa High School,District,2949,1884411,81.158020,76.711767,65.988471,80.739234,73.363852,24649428,Medium
2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,93.867121,95.854628,94.860875,24649428,Small
3,Hernandez High School,District,4635,3022020,80.934412,77.289752,66.752967,80.862999,73.807983,24649428,Large
4,Griffin High School,Charter,1468,917500,83.816757,83.351499,93.392371,97.138965,95.265668,24649428,Small
5,Wilson High School,Charter,2283,1319574,83.989488,83.274201,93.867718,96.539641,95.203679,24649428,Medium
6,Cabrera High School,Charter,1858,1081356,83.975780,83.061895,94.133477,97.039828,95.586652,24649428,Small
7,Bailey High School,District,4976,3124928,81.033963,77.048432,66.680064,81.933280,74.306672,24649428,Large
8,Holden High School,Charter,427,248087,83.814988,83.803279,92.505855,96.252927,94.379391,24649428,Small
9,Pena High School,Charter,962,585858,84.044699,83.839917,94.594595,95.945946,95.270270,24649428,Small


In [187]:
# isolate pertinent columns
reduced = withsize[['size',
                    'Average Math Score', 
                    'Average Reading Score', 
                    '% Passing Math', 
                    '% Passing Reading', 
                    '% Passing Overall']]
reduced

,size,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,Medium,76.629414,81.182722,65.683922,81.316421,73.500171
1,Medium,76.711767,81.158020,65.988471,80.739234,73.363852
2,Small,83.359455,83.725724,93.867121,95.854628,94.860875
3,Large,77.289752,80.934412,66.752967,80.862999,73.807983
4,Small,83.351499,83.816757,93.392371,97.138965,95.265668
5,Medium,83.274201,83.989488,93.867718,96.539641,95.203679
6,Small,83.061895,83.975780,94.133477,97.039828,95.586652
7,Large,77.048432,81.033963,66.680064,81.933280,74.306672
8,Small,83.803279,83.814988,92.505855,96.252927,94.379391
9,Small,83.839917,84.044699,94.594595,95.945946,95.270270


In [188]:
# groupby bins to produce summary
grouped = reduced.groupby('size')

In [197]:
# calculate averages by bin for each column
output3 = grouped.mean()

In [198]:
# format for readability
output4 = output3.style.format({'Average Math Score':'{:.1f}', 
                                 'Average Reading Score':'{:.1f}', 
                                 '% Passing Math':'{:.1f}%', 
                                 '% Passing Reading':'{:.1f}%', 
                                 '% Passing Overall':'{:.1f}%'})
output4

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
size,,,,,
Small,83.5,83.9,93.6%,96.6%,95.1%
Medium,78.4,81.8,73.5%,84.5%,79.0%
Large,77.1,80.9,66.5%,81.1%,73.8%


#### School performance by type

In [191]:
renamed.head()

,School,Type,Total Students,School Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Passing Overall,Total Budget
0,Huang High School,District,2917,1910635,81.182722,76.629414,65.683922,81.316421,73.500171,24649428
1,Figueroa High School,District,2949,1884411,81.158020,76.711767,65.988471,80.739234,73.363852,24649428
2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,93.867121,95.854628,94.860875,24649428
3,Hernandez High School,District,4635,3022020,80.934412,77.289752,66.752967,80.862999,73.807983,24649428
4,Griffin High School,Charter,1468,917500,83.816757,83.351499,93.392371,97.138965,95.265668,24649428


In [192]:
# isolate pertinent columns
summarysubset = renamed[['Type',
                         'Average Math Score', 
                         'Average Reading Score', 
                         '% Passing Math', 
                         '% Passing Reading', 
                         '% Passing Overall']]
summarysubset

,Type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,District,76.629414,81.182722,65.683922,81.316421,73.500171
1,District,76.711767,81.158020,65.988471,80.739234,73.363852
2,Charter,83.359455,83.725724,93.867121,95.854628,94.860875
3,District,77.289752,80.934412,66.752967,80.862999,73.807983
4,Charter,83.351499,83.816757,93.392371,97.138965,95.265668
5,Charter,83.274201,83.989488,93.867718,96.539641,95.203679
6,Charter,83.061895,83.975780,94.133477,97.039828,95.586652
7,District,77.048432,81.033963,66.680064,81.933280,74.306672
8,Charter,83.803279,83.814988,92.505855,96.252927,94.379391
9,Charter,83.839917,84.044699,94.594595,95.945946,95.270270


In [200]:
# groupby school type to calculate averages for each type
typegroup = summarysubset.groupby('Type')
output5 = typegroup.mean()

In [201]:
# format for readability
output6 = output5.style.format({'Average Math Score':'{:.1f}', 
                                'Average Reading Score':'{:.1f}', 
                                '% Passing Math':'{:.1f}%', 
                                '% Passing Reading':'{:.1f}%', 
                                '% Passing Overall':'{:.1f}%'})
output6

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Type,,,,,
Charter,83.5,83.9,93.6%,96.6%,95.1%
District,77.0,81.0,66.5%,80.8%,73.7%
